In [1]:
#importstatements
import matplotlib.pyplot as plt
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as F     

from pyspark.sql import types 
from pyspark.sql.types import StructField, StringType, LongType, DoubleType, BooleanType, StructType, IntegerType

In [2]:
#config for our sparksession
config = pyspark.SparkConf().setAll([
    ('spark.executor.memory', '16g'), 
    ('spark.executor.cores', '4'), 
    ('spark.cores.max', '8'),
    ('spark.driver.memory','2g'),
    ('spark.executor.instances', '1'),
    ('spark.dynamicAllocation.enabled', 'true'),
    ('spark.dynamicAllocation.shuffleTracking.enabled', 'true'),
    ('spark.dynamicAllocation.executorIdleTimeout', '60s'),
    ('spark.dynamicAllocation.minExecutors', '2'),
    ('spark.dynamicAllocation.maxExecutors', '2'),
    ('spark.dynamicAllocation.initialExecutors', '1'),
    ('spark.dynamicAllocation.executorAllocationRatio', '1'),
    ('spark.worker.cleanup.enabled', 'true'),
    ('spark.worker.cleanup.interval', '60'),
    ('spark.shuffle.service.db.enabled', 'true'),
    ('spark.worker.cleanup.appDataTtl', '60rite'),
    ('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.2')
])

In [3]:
#create sparksession
#when copying change appName
spark = SparkSession \
    .builder \
    .config(conf=config) \
    .appName("7_TransactionUseCases") \
    .master("spark://172.23.149.212:7077") \
    .getOrCreate()

22/07/13 16:30:42 WARN Utils: Your hostname, algorand-druid-and-spark resolves to a loopback address: 127.0.0.1; using 172.23.149.212 instead (on interface ens3)
22/07/13 16:30:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.mongodb.spark#mongo-spark-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-eedce58f-4095-431d-8338-e4dec9f4bb31;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector;10.0.2 in central
	found org.mongodb#mongodb-driver-sync;4.5.1 in central
	[4.5.1] org.mongodb#mongodb-driver-sync;[4.5.0,4.5.99)
	found org.mongodb#bson;4.5.1 in central
	found org.mongodb#mongodb-driver-core;4.5.1 in central
:: resolution report :: resolve 3353ms :: artifacts dl 4ms
	:: modules in use:
	org.mongodb#bson;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-core;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-sync;4.5.1 from central in [default]
	org.mongodb.spark#mongo-spark-connector;10.0.2 from central in [default]
	---------------------------------------------------------------------
	|                  |  

In [4]:
#create a schema so all data quality is ensured
schema = StructType([ \
    StructField("_id", StringType(), True), \
    StructField("asset", LongType(), True), \
    StructField("extra", StringType(), True), \
    StructField("intra", LongType(), True), \
    StructField("round", LongType(), True), \
    StructField("rr", LongType(), True), \
    StructField("sig", StringType(), True), \
    StructField("txid", StringType(), True), \
    StructField("txn_aamt", LongType(), True), \
    StructField("txn_aclose", StringType(), True), \
    StructField("txn_afrz", BooleanType(), True), \
    StructField("txn_amt", LongType(), True), \
    StructField("txn_apaa", StringType(), True), \
    StructField("txn_apan", LongType(), True), \
    StructField("txn_apap", StringType(), True), \
    StructField("txn_apar", StringType(), True), \
    StructField("txn_apas", StringType(), True), \
    StructField("txn_apat", StringType(), True), \
    StructField("txn_apep", StringType(), True), \
    StructField("txn_apfa", StringType(), True), \
    StructField("txn_apgs", StringType(), True), \
    StructField("txn_apid", LongType(), True), \
    StructField("txn_apls", StringType(), True), \
    StructField("txn_apsu", StringType(), True), \
    StructField("txn_arcv", StringType(), True), \
    StructField("txn_asnd", StringType(), True), \
    StructField("txn_caid", LongType(), True), \
    StructField("txn_close", StringType(), True), \
    StructField("txn_fadd", StringType(), True), \
    StructField("txn_faid", LongType(), True), \
    StructField("txn_fee", LongType(), True), \
    StructField("txn_fv", LongType(), True), \
    StructField("txn_gen", StringType(), True), \
    StructField("txn_gh", StringType(), True), \
    StructField("txn_grp", StringType(), True), \
    StructField("txn_lsig", StringType(), True), \
    StructField("txn_lv", LongType(), True), \
    StructField("txn_lx", StringType(), True), \
    StructField("txn_msig", StringType(), True), \
    StructField("txn_nonpart", BooleanType(), True), \
    StructField("txn_note", StringType(), True), \
    StructField("txn_rcv", StringType(), True), \
    StructField("txn_rekey", StringType(), True), \
    StructField("txn_selkey", StringType(), True), \
    StructField("txn_sig", StringType(), True), \
    StructField("txn_snd", StringType(), True), \
    StructField("txn_type", StringType(), True), \
    StructField("txn_votefst", LongType(), True), \
    StructField("txn_votekd", LongType(), True), \
    StructField("txn_votekey", StringType(), True), \
    StructField("txn_votelst", LongType(), True), \
    StructField("txn_xaid", LongType(), True), \
    StructField("typeenum", IntegerType(), True) \
])

In [5]:
# account table to determine which accounts have received rewards
dfTx = spark.read.format("mongodb") \
    .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.212:27017') \
    .option('spark.mongodb.database', 'algorand') \
    .option('spark.mongodb.collection', 'txn') \
    .option('park.mongodb.read.readPreference.name', 'primaryPreferred') \
    .option('spark.mongodb.change.stream.publish.full.document.only','true') \
    .option("forceDeleteTempCheckpointLocation", "true") \
    .schema(schema) \
    .load()


In [6]:
#drop all unnecessary tables
dfTx = dfTx.select("typeenum", "round", "txn_snd", "txn_type", "txn_apid", "txn_apan", "txn_apas", "txn_apap", "txid")

In [8]:
#use case based grouping
#1-> payment transaction
#2-> Keyreg transaction
#3-> asset configuration
#4-> asset transfer
#5-> asset freeze
#6-> application transactions
dfTx = dfTx.withColumn("usecase", F.when(F.col('typeenum')== 1, "payment")
                       .when(F.col('typeenum')== 2, "keyreg")
                       .when(F.col('typeenum')== 3, "assetconfig")
                       .when(F.col('typeenum')== 4, "assettransfer")
                       .when(F.col('typeenum')== 5, "assetfreeze")
                       .when(F.col('typeenum')== 6, "application"))
                       

In [10]:
dfTxGroupUseCases = dfTx.groupBy("usecase").count()

In [11]:
#newest round has to be calculated
newestRound = dfTx.agg(F.max("round")).collect()[0][0]
dfTxGroupUseCasesGold = dfTxGroupUseCases.withColumn("CreationRound", F.lit(newestRound))

In [ ]:
dfTxGroupUseCasesGold.write.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.212:27017') \
  	.mode("append") \
    .option('spark.mongodb.database', 'algorand_gold') \
  	.option('spark.mongodb.collection', 'TransactionsByUseCase_7') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
  	.save()

In [ ]:
#collect so a python object is created
graph = dfTxGroupUseCases.collect()

In [ ]:

#convert row["data"] to only data
UCnames = [row[0] for (row) in graph]
UCvalues = [row[1] for (row) in graph]


In [ ]:
plt.figure()
for i in range(len(UCnames)):
    plt.bar(UCnames[i], UCvalues[i], width = 0.4)
    
plt.title("Smart Contract Use Case Transactions", loc ='center', pad = None)
plt.savefig('/home/ubuntu/apps/figures/7_TransactionUC/TC_Use_Cases.jpg', dpi= 200)
plt.show()
plt.close()